# Klasyczne Szyfry Mechaniczne

### Literatura:
1. https://www.cryptomuseum.com/crypto/enigma/wiring.htm
2. http://users.telenet.be/d.rijmenants/en/enigmatech.htm#wiringdiagram
3. http://rumkin.com/tools/cipher/playfair.php
4. https://archive.org/details/cryptanalysis00hele/mode/2up
5. https://www.codesandciphers.org.uk/lorenz/fish.htm
6. https://www.thehistorypress.co.uk/articles/how-lorenz-was-different-from-enigma/

## Enigma
---
Enigma (z gr. αινιγμα zagadka) – niemiecka przenośna elektromechaniczna maszyna szyfrująca, oparta na mechanizmie obracających się wirników, skonstruowana przez Artura Scherbiusa.
Enigma znana i stosowana była już od lat 20. XX wieku. Początkowo wykorzystywano ją jedynie w celach komercyjnych. Natomiast w czasach II Wojny Światowej zaadaptowana została przez niemieckie siły zbrojne oraz inne służby państwowe i wywiadowcze. Co warte podkreślenia Enigma wykorzystywana była również przez inne państwa.

In [4]:
# kod enigmy opisujący działanie modelu komercyjnego "Enigma K (A27)" z roku 1927
# model ten posiadał 3 rotory i ustawialny reflektor

# importujemy moduł, który pozwoli nam łatwiej kopiować tablice
import copy

# pomocniczo zapisujemy sobie alfabet
alphabet    = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# zmienne opisujące aktualne i pierwotne ustawienie rotorów
# pierwotne ustawienie jest przechowywane w celach deszyfrowania
curr_rotor_setup = [0, 0, 0]
init_rotor_setup = [0, 0, 0]

# statyczny połączenia wejściowe określające pierwotną zamianę parami
# liter alfabetu na początku i na końcu procesu szyfrowania
entry_wiring = {
    "A": "Q", "B": "W", "C": "E", "D": "R", "E": "C",
    "F": "T", "G": "Z", "H": "U", "I": "O", "J": "S",
    "K": "Y", "L": "P", "M": "V", "N": "X", "O": "I",
    "P": "L", "Q": "A", "R": "D", "S": "J", "T": "F",
    "U": "H", "V": "M", "W": "B", "X": "N", "Y": "K",
    "Z": "G" }

# rotory to permutacje alfabetu wykorzystywane w procesie szyfrowania
rotor_IC    = "DMTWSILRUYQNKFEJCAZBPGXOHV"
rotor_IIC   = "HQZGPJTMOBLNCIFDYAWVEUSRKX"
rotor_IIIC  = "UQNTLSZFMREHDPXKIBVYGJCWOA"

# zmienna ułatwiająca nam pracę na rotorach (i określająca ich ułożenie)
rotors = [rotor_IC, rotor_IIC, rotor_IIIC]

# reflektor określa jak litery zostają zamienione w środkowej fazie szyfrowania,
# litery dobrane są parami, litera nie może zamienić się w samą siebie
reflector = {
    "A": "V", "B": "M", "C": "E", "D": "F", "E": "C",
    "F": "D", "G": "J", "H": "O", "I": "W", "J": "G",
    "K": "Q", "L": "P", "M": "B", "N": "U", "O": "H",
    "P": "L", "Q": "K", "R": "Z", "S": "Y", "T": "X",
    "U": "N", "V": "A", "W": "I", "X": "T", "Y": "S",
    "Z": "R" }

def turn_rotors(rotor_setup):
    # pierwszy rotor zawsze się przestawia
    rotor_setup[0] += 1
    # z powodów technicznych "zawijamy" wartość zmiennej
    rotor_setup[0] %= 26

    if rotor_setup[0] == 17:
        # jeżeli pierwszy rotor dotrze do pozycji "R" (w alfabecie), 
        # to przestawiamy rotor drugi
        rotor_setup[1] += 1
        # z powodów technicznych "zawijamy" wartość zmiennej
        rotor_setup[1] %= 26
        if rotor_setup[1] == 12:
            # jeżeli drugi rotor dotrze do pozycji "M" (w alfabecie), 
            # to przestawiamy rotor trzeci
            rotor_setup[2] += 1
            # z powodów technicznych "zawijamy" wartość zmiennej
            rotor_setup[2] %= 26
    
    return rotor_setup

def flip_entry(letter):
    # zwróć odpowiednią literę według podłączenia wejściowego
    return entry_wiring[letter]

def reflect(letter):
    # zwróć odpowiednią literę według podłączenia w reflektorze
    return reflector[letter]

# funkcja opisująca szyfrowanie każdej litery przez rotory przy "wchodzeniu",
# to jest przed odbiciem na reflektorze
def rotate_letter_entry(letter, rotor_setup):
    # litera po kolei przechodzi przez trzy rotory w kolejności 1, 2, 3
    # na każdym zostaje zamieniona zgodnie z ustawieniem rotora
    for i in range(len(rotors)):
        letter = rotors[i][(alphabet.index(letter) + rotor_setup[i]) % 26]
    return letter

# funkcja opisująca szyfrowanie każdej litery przez rotory przy "wchodzeniu",
# to jest po odbiciu na reflektorze
def rotate_letter_exit(letter, rotor_setup):
    # litera przechodzi przez trzy rotory w odwróconej kolejności - 3, 2, 1,
    # na każdym zostaje zamieniona zgodnie z ustawieniem rotora
    for i in range(len(rotors) - 1, -1, -1):
        letter = alphabet[(rotors[i].index(letter) - rotor_setup[i]) % 26]
    return letter

# sama funkcja opisująca proces szyfrowania/deszyfrowania litery
def encrypt(letter, rotor_setup):
    # litera zostaje zamieniona wg podłączenia wejściowego
    letter = flip_entry(letter)
    
    # litera przechodzi przez rotory w kolejności 1, 2, 3
    letter = rotate_letter_entry(letter, rotor_setup)

    # litera jest odbijana na reflektorze
    letter = reflect(letter)
    
    # litera przechodzi ponownie przez rotory,
    # tym razem w odwrotnej kolejności - 3, 2, 1
    letter = rotate_letter_exit(letter, rotor_setup)

    # litera zostaje ponownie zamieniona wg podłączenia wejściowego
    letter = flip_entry(letter)

    # zwrot zaszyfrowanej litery
    return letter

# kod wprowadzający
if __name__ == "__main__":
    # dla każdego rotora
    for i in range(len(rotors)):
        # pobieramy od użytkownika ustawienie początkowe
        curr_rotor_setup[i] = int(input(f"Podaj ustawienie rotora nr {i+1} (liczba w zakresie 0-25): "))

    # tworzymy kopię tablicy przechowującej ustawienia rotorów
    # będzie nam potrzebna do deszyfrowania
    init_rotor_setup = copy.deepcopy(curr_rotor_setup)
        
    # pobieramy od użytkownika plaintext do zaszyfrowania i zamieniamy na wielkie litery
    plaintext = input("plaintext (litery, bez znaków specjalnych i cyfr): ").upper()
    # pozbywamy się przerw między słowami
    plaintext = ''.join(plaintext.split())

    # zmienna przechowująca szyfrogram
    cyphertext = ""
    # dla każdej litery w plaintext
    for letter in plaintext: 
        # szyfrujemy ją na podstawie "aktualnego" ustawienia rotorów
        cyphertext += encrypt(letter, curr_rotor_setup)
        # po zaszyfrowaniu litery obracamy rotory
        curr_rotor_setup = turn_rotors(curr_rotor_setup)

    # wypisujemy szyfrogram
    print(f"Cyphertext: {cyphertext}")

    # zmienna przechowująca tekst odszyfrowany
    decodedtext = ""
    # dla każdej litery w szyfrogramie
    for letter in cyphertext:
        # w celu odszyfrowania należy każdą z liter ponownie "zaszyfrować"
        # zgodnie z "pierwotnym" ustawieniem rotorów, które kopiowaliśmy
        decodedtext += encrypt(letter, init_rotor_setup)
        # po odszyfrowaniu litery obracamy rotory
        init_rotor_setup = turn_rotors(init_rotor_setup)

    # wypisujemy zdekodowany tekst
    print(f"Decoded text: {decodedtext}")
    # powinien się pokryć z tekstem oryginalnym
    # (chociaż nie ma spacji i nie rozróżnia małych/wielkich liter)

Podaj ustawienie rotora nr 1 (liczba w zakresie 0-25): 5
Podaj ustawienie rotora nr 2 (liczba w zakresie 0-25): 5
Podaj ustawienie rotora nr 3 (liczba w zakresie 0-25): 5
plaintext (litery, bez znaków specjalnych i cyfr): TEST TEST TEST
Cyphertext: ADTGPZRHAKYH
Decoded text: TESTTESTTEST


## Playfair
---
Szyfr Playfair (czasem nazywany też szyfrem Playfaira) został wymyślony przez sir Charlesa Wheatstone'a w 1854, a spopularyzowany przez barona Lyona Playfaira.

In [6]:
import copy
import numpy as np

# funkcja pomocnicza, która zamienia nam string na listę znaków spełniających zasady
# tworzenia klucza - unikalne, J zostaje zamienione na I, kolejność zostaje zachowana
def process_key(string):
    unique = []
    # iterujemy po znakach w stringu
    for char in string:
        # jeśli znak to 'J' zamień je na 'I'
        if char == 'J':
            char = 'I'
        # jeśli znak jeszcze nie wystąpił oraz jest literą
        if (char not in unique) and (char.isalpha()):
            # dodaj go do listy unikalnych znaków
            unique.append(char)

    # zwróć klucz
    return unique

# funkcja pomocniczna przetwarzająca string na pary liter
# przygotowane do szyfrowania/deszyfrowania
def process_text(string):
    # zmienna przechowująca wszystkie pary
    all_pairs = []
    # aktualnie budowana para
    curr_pair = []
    # iterujemy po znakach w stringu
    for char in string:
        # pozbywamy się litery 'J', zamieniamy ją na 'I'
        if char == 'J':
            char = 'I'
        # jeśli znak jest literą
        if char.isalpha():
            # jeśli w aktualnie budowanej parze jest jedna litera
            if len(curr_pair) == 1:
                # sprawdzamy, czy litera się powtarza (taka para jest nieszyfrowalna)
                if char == curr_pair[0]:
                    # jeżeli tak, to dopisujemy 'X' do pary
                    curr_pair.append('X')
                    # dodajemy ją do listy gotowych par
                    all_pairs.append(copy.copy(curr_pair))
                    # a naszą aktualną literę umieszczamy w nowej parze
                    curr_pair = []
                    curr_pair.append(char)
                    # i przechodzimy do następnej litery
                    continue
            # dopisujemy naszą literę do aktualnie budowanej pary
            curr_pair.append(char)
            # jeśli nasza para ma długość 2 (jest pełna)
            if len(curr_pair) == 2:
                # dodajemy ją do listy gotowych par
                all_pairs.append(copy.copy(curr_pair))
                # opróżniamy roboczą parę
                curr_pair = []

    # kiedy już sprawdzimy wszystkie litery, sprawdzamy czy była ich parzysta ilość
    # (przez sprawdzenie czy nie została nam niedokończona para)
    if len(curr_pair) == 1:
        # jeżeli tak, to dodajemy do niej 'X'
        curr_pair.append('X')
        # a następnie dodajemy ją do listy gotowych par
        all_pairs.append(copy.copy(curr_pair))
        
    # zwracamy listę gotowych par
    return all_pairs

# funkcja pomocnicza do tworzenia siatki szyfrowania na podstawie klucza
def create_grid(key):
    # alfabet bez litery J
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    # tworzymy roboczą kopię tablicy klucza
    used = copy.deepcopy(key)
    # tworzymy pustą tablicę o 5 rzędach
    grid = [ [], [], [], [], [] ]
    # licznik aktualnej pozycji w kluczu
    key_ctr = 0
    # licznik aktualnej pozycji w alfabecie
    alph_ctr = 0

    # licznik rzędów
    for row in range(5):
        # licznik kolumn
        for _ in range(5):
            # jeżeli jeszcze nie wpisaliśmy wszystkich liter z klucza
            if key_ctr < len(key):
                # dopisujemy do aktualnego rzędu aktualną literę z klucza
                grid[row].append(used[key_ctr])
                # inkrementujemy licznik klucza
                key_ctr += 1
            else:
                # jeżeli wszystkie litery z klucza są już wpisane
                # przesuwamy licznik alfabetu aż trafimy na literę
                # jeszcze nie dodaną do siatki
                while alphabet[alph_ctr] in used:
                    alph_ctr += 1
                # dopisujemy literę alfabetu do siatki i listy zużytych liter
                grid[row].append(alphabet[alph_ctr])
                used.append(alphabet[alph_ctr])

    # zwracamy siatkę jako obiekt klasy array z biblioteki numpy
    return np.array(grid)

# funkcja szyfrująca
def encrypt(pairs, grid):
    # zmienna przechowująca szyfrogram
    encrypted = ""
    # dla każdej pary (przygotowany tekst)
    for pair in pairs:
        # korzystając z funkcji numpy znajdujemy koordynaty liter w siatce
        coords = []
        coords.append(list(np.argwhere(grid == pair[0])[0]))
        coords.append(list(np.argwhere(grid == pair[1])[0]))
        
        # sprawdzamy czy litery z pary są w tym samym rzędzie
        if coords[0][0] == coords[1][0]:
            # jeśli tak to zastępujemy je literami o 1 pozycję na prawo od nich
            encrypted += grid[coords[0][0]][(coords[0][1] + 1) % 5]
            encrypted += grid[coords[1][0]][(coords[1][1] + 1) % 5]
            encrypted += " "

        # jeżeli nie to sprawdzamy czy litery z pary są w tej samej kolumnie
        elif coords[0][1] == coords[1][1]:
            # jeśli tak to zastępujemy je literami o 1 pozycję w dół od nich
            encrypted += grid[(coords[0][0] + 1) % 5][coords[0][1]]
            encrypted += grid[(coords[1][0] + 1) % 5][coords[1][1]]
            encrypted += " "

        # w takim razie muszą tworzyć prostokąt
        else:
            # zamieniamy litery tak, aby litera została zastąpiona tą
            # z 'niezajętego' kąta prostokąta, ale w tym samym rzędzie
            encrypted += grid[coords[0][0]][coords[1][1]]
            encrypted += grid[coords[1][0]][coords[0][1]]
            encrypted += " "

    # zwracamy string zaszyfrowanych par
    return encrypted

# funkcja deszyfrująca
def decrypt(pairs, grid):
    decrypted = ""
    for pair in pairs:
        coords = []
        coords.append(list(np.argwhere(grid == pair[0])[0]))
        coords.append(list(np.argwhere(grid == pair[1])[0]))
        
        # sprawdzamy czy litery z pary są w tym samym rzędzie
        if coords[0][0] == coords[1][0]:
            # jeśli tak to zastępujemy je literami o 1 pozycję na lewo od nich
            # (odwrotność szyfrowania)
            decrypted += grid[coords[0][0]][(coords[0][1] - 1) % 5]
            decrypted += grid[coords[1][0]][(coords[1][1] - 1) % 5]
            decrypted += " "

        # jeżeli nie to sprawdzamy czy litery z pary są w tej samej kolumnie
        elif coords[0][1] == coords[1][1]:
            # jeśli tak to zastępujemy je literami o 1 pozycję w górę od nich
            # (odwrotność szyfrowania)
            decrypted += grid[(coords[0][0] - 1) % 5][coords[0][1]]
            decrypted += grid[(coords[1][0] - 1) % 5][coords[1][1]]
            decrypted += " "

        # w takim razie muszą tworzyć prostokąt
        else:
            # zamieniamy litery tak, aby litera została zastąpiona tą
            # z 'niezajętego' kąta prostokąta, ale w tym samym rzędzie
            # (tak samo jak w szyfrowaniu)
            decrypted += grid[coords[0][0]][coords[1][1]]
            decrypted += grid[coords[1][0]][coords[0][1]]
            decrypted += " "

    # zwracamy string zdeszyfrowanych par
    return decrypted

# obsługa wejścia
if __name__ == '__main__':
    # pobranie od użytkownika klucza i przygotowanie go do pracy
    key = input('Podaj klucz (litery, bez cyfr): ')
    key = process_key(key.upper())

    # utworzenie z klucza siatki szyfrowej
    pf_grid = create_grid(key)

    # pobranie od użytkownika plaintextu i przygotowanie go do pracy
    plaintext = input('Podaj plaintext (litery, bez cyfr): ').upper()
    print(f'Plaintext: {plaintext}')
    plaintext = process_text(plaintext)

    # zaszyfrowanie podanej wiadomości i pokazanie wyników
    cyphertext = encrypt(plaintext, pf_grid)
    print(f'Tekst zaszyfrowany: {cyphertext}')

    # przygotowanie szyfrogramu do rozszyfrowania
    to_decrypt = process_text(cyphertext)

    # zdeszyfrowanie szyfrogramu i pokazanie wyników
    decryptedtext = decrypt(to_decrypt, pf_grid)
    print(f'Tekst zdeszyfrowany: {decryptedtext}')


Podaj klucz (litery, bez cyfr): playfair example
Podaj plaintext (litery, bez cyfr): hide the gold in the tree stump
Plaintext: HIDE THE GOLD IN THE TREE STUMP
Tekst zaszyfrowany: BM OD ZB XD NA BE KU DM UI XM MO UV IF 
Tekst zdeszyfrowany: HI DE TH EG OL DI NT HE TR EX ES TU MP 


## Maszyna Lorenza
---
Maszyna Lorenza (Lorenz-Chiffre, Schlüsselzusatz; Lorenz SZ 40 i SZ 42) – niemiecka maszyna szyfrująca używana podczas II wojny światowej dla przekazu informacji przez dalekopisy. Brytyjscy analitycy, którzy zakodowane komunikaty dalekopisowe określali kodem "Fish" (ryba), szyfry maszyny Lorenza i ją samą nazywali "Tunny" (tuńczyk). O ile Enigma była używana przez jednostki polowe, o tyle Tunny służyła do komunikacji wysokiego szczebla, gdzie można było wykorzystać ciężką maszynę, dalekopis i oddzielne łącza. 

In [9]:
# UWAGA - W kodzie znajdują się elementy niezaimplementowanej
# funkcjonalności - obsługi znaków specjalnych.
# O ile kodowanie ITA2 pozwala na ich obsługę, nie byłem w stanie
# znaleźć informacji odnośnie tego jak sama maszyna je obsługiwała.
# Ponieważ pełna obsługa znaków specjalnych znacznie i w mojej opinii
# niepotrzebnie komplikowałaby kod demonstracyjny,
# postanowiłem jej nie dokańczać ~ JD


import copy

# definicje rotorów odpowiednich grup, każdy innej długości,
# normalnie kombinacje 0 i 1 były konfigurowane przez użytkowników
# jednak na potrzeby prezentacji zostały wygenerowane losowo i są statyczne

# rotory psi
rotor_psi1  = '0110100100111100010010010111010001011111011'
rotor_psi2  = '10101101011100001100101011001001100011011101001'
rotor_psi3  = '111110110111110001110011001001001010011111001100010'
rotor_psi4  = '11010110111001100011111110011101011010001001011100111'
rotor_psi5  = '10000001000101001010110001010111110111110111100100011010111'

rotor_psi_list  = [rotor_psi1, rotor_psi2, rotor_psi3, rotor_psi4, rotor_psi5]

# rotory 
rotor_mu1   = '0010110100111110010010011011000110000'
rotor_mu2   = '0011110011101000001011011100100001011010000101001011101111101'

rotor_mu_list   = [rotor_mu1, rotor_mu2]

# rotory chi obracały się z każdą literą
rotor_chi1  = '00010001010110110010110011001101011011001'
rotor_chi2  = '0001011011111100111000111101100'
rotor_chi3  = '01000111101101010101010110000'
rotor_chi4  = '01111100011000001000010001'
rotor_chi5  = '01001100010000010110111'

rotor_chi_list  = [rotor_chi1, rotor_chi2, rotor_chi3, rotor_chi4, rotor_chi5]

# Zmienna przechowująca ilość możliwych ustawień danych rotorów w grupach
rotor_psi_len   = [43, 47, 51, 53, 59]
rotor_mu_len    = [37, 61]
rotor_chi_len   = [41, 31, 29, 26, 23]

# Tworzenie zmiennych przechowujących aktualne ustawienia rotorów konkretnych grup
# Utworzone "losowe" ustawienie na potrzeby demonstracji
rotor_psi_curr_setup    = [6, 4, 4, 5, 1]
rotor_mu_curr_setup     = [2, 3]
rotor_chi_curr_setup    = [1, 7, 4, 8, 6]

# Tworzenie zmiennych przechowujących pierwotne ustawienia rotorów konkretnych grup
rotor_psi_init_setup    = [0, 0, 3, 0, 0]
rotor_mu_init_setup     = [0, 0]
rotor_chi_init_setup    = [0, 0, 1, 0, 0]

# pomocniczy słownik do mapowania kodów ita2 na litery
ita2_to_letter = {
    '00000': '@', # zastąpienie znaku NULL na potrzeby prezentacji
    '11111': '#', # zastąpienie znaku ZMIANA TRYBU NA LITEROWY na potrzeby prezentacji
    '00010': '<', # zastąpienie znaku CR na potrzeby prezentacji
    '01000': '>', # zastąpienie znaku LF na potrzeby prezentacji
    '00100': ' ',   '11101': 'Q',   '11001': 'W',  
    '10000': 'E',   '01010': 'R',   '00001': 'T',   '10101': 'Y',   '11100': 'U',
    '01100': 'I',   '00011': 'O',   '01101': 'P',   '11000': 'A',   '10100': 'S',
    '10010': 'D',   '10110': 'F',   '01011': 'G',   '00101': 'H',   '11010': 'J',
    '11110': 'K',   '01001': 'L',   '10001': 'Z',   '10111': 'X',   '01110': 'C',
    '01111': 'V',   '10011': 'B',   '00110': 'N',   '00111': 'M',
    '11011': '%' # zastąpienie znaku ZMIANA TRYBU NA SYMBOLOWY na potrzeby prezentacji
}

# [NIEUŻYWANY] pomocniczy słownik do mapowania kodów ita2 na znaki
ita2_to_special = {
    '00010': '\r',  '01000': '\n',  '00100': ' ',   '11101': '1',   '11001': '2',
    '10000': '3',   '01010': '4',   '00001': '5',   '10101': '6',   '11100': '7',
    '01100': '8',   '00011': '9',   '01101': '0',   '11000': '-',   '10100': '\'',
    '10010': '$',   '10110': '!',   '01011': '&',   '00101': '#',   '11010': '£', 
    '11110': '(',   '01001': ')',   '10001': '+',   '10111': '/',   '01110': ':',  
    '01111': '=',   '10011': '?',   '00110': ',',   '00111': '.',   '11111': '_LS'
}

# lista znaków dozwolonych, z której korzysta funkcja czyszcząca tekst
allowed_chars = list('\r\n QWERTYUIOPASDFGHJKLZXCVBNM') # [OGRANICZENIE FUNKCJONALNOŚCI]
# allowed_chars = list('\r\n QWERTYUIOPASDFGHJKLZXCVBNM1234567890-\'$!&#£()+/:=?,.')

# pomocniczy słownik do mapowania liter na kody ita2 (odwrócony słownik ita2_to_letter)
letter_to_ita2 = {value : key for (key, value) in ita2_to_letter.items()}

# [NIEUŻYWANY] pomocniczy słownik do mapowania znaków na kody ita2 (odwrócony słownik ita2_to_special)
special_to_ita2 = {value : key for (key, value) in ita2_to_special.items()}

# funkcja pomocnicza do "XORowania" znaków "1" i "0"
def xor_mock_binary(bit0, bit1):
    # Jeżeli "bity" są identyczne zwróć "0"
    if bit0 == bit1:
        return '0'
    # w przeciwnym razie zwróć "1"
    else:
        return '1'

# funkcja pomocnicza czyszcząca tekst ze znaków nieobsługiwanych przez maszynę
def sanitize_text(string):
    # zmienna przechowująca "przygotowany" tekst
    sanitized = ''
    # iterujemy po każdej literze stringa
    for char in string.upper():
        # jeśli znak jest alfanumeryczny
        if char in allowed_chars:
            # dodajemy go do naszego stringa zatwierdzonych znaków
            sanitized += char

    return sanitized

# funkcja odpowiadająca za obrót rotorów
def shift_rotors(psi, mu, chi):
    # rotory chi zawsze się obracają
    for i in range(len(chi)):
        chi[i] += 1
        chi[i] %= rotor_chi_len[i]
    
    # rotor mu1 zawsze się obraca
    mu[0] += 1
    mu[0] %= rotor_mu_len[0]

    # rotor mu2 obraca się tylko jeśli mu1 jest ustawiony na wartość '1'
    if rotor_mu1[mu[0]] == '1':
        mu[1] += 1
        mu[1] %= rotor_mu_len[1]

    # rotory psi obracają się tylko jeśli mu2 jest ustawiony na wartość '1'
    if rotor_mu2[mu[1]] == '1':
        for i in range(len(psi)):
            psi[i] += 1
            psi[i] %= rotor_psi_len[i]
    
    return (psi, mu, chi)

# funkcja pomagająca wykonać XORowanie z kluczem danego znaku w ITA2
def xor_ita2(code, psi, mu, chi):
    res_code = ''
    # bit po bicie XOrujemy z odpowiednimi rotorami
    for i in range(len(code)):
        res = xor_mock_binary(code[i], rotor_chi_list[i][chi[i]])
        res = xor_mock_binary(res, rotor_psi_list[i][psi[i]])
        res_code += res

    # obracamy rotory po wykonaniu pracy na danym znaku
    psi, mu, chi = shift_rotors(psi, mu, chi)

    return (res_code, psi, mu, chi)

# funkcja szyfrująca (deszyfrowanie przez ponowne szyfrowanie z tymi samymi parametrami)
def encrypt(string, psi, mu, chi):
    # zmienna przechowująca tekst zaszyfrowany
    cyphertext = ''
    for char in string:
        # zamieniamy literę na jej kod ITA2
        ita = letter_to_ita2[char]
        # wykonujemy XORowanie z naszym 'kluczem' (odpowiednio ustawione rotory)
        ita, psi, mu, chi = xor_ita2(ita, psi, mu, chi)
        # dopisujemy wynik w postaci czytelnej (na potrzeby demonstracji) do cyphertextu
        cyphertext += ita2_to_letter[ita]
    return cyphertext

# obsługa wejścia
if __name__ == '__main__':
    # ============================================================== #
    # V # ODKOMENTOWYWAĆ NA WŁASNĄ ODPOWIEDZIALNOŚĆ - MĘCZĄCE!!! # V #
    # ============================================================== #

    # # pobranie od użytkownika pierwotnych ustawień rotorów
    # for i in range(len(rotor_psi_curr_setup)):
    #     rotor_psi_curr_setup[i] = int(input(f"Podaj ustawienie rotora psi{i+1} (liczba w zakresie 0-{rotor_psi_len[i]}): "))

    # for i in range(len(rotor_mu_curr_setup)):
    #     rotor_mu_curr_setup[i] = int(input(f"Podaj ustawienie rotora mu{i+1} (liczba w zakresie 0-{rotor_mu_len[i]}): "))

    # for i in range(len(rotor_chi_curr_setup)):
    #     rotor_chi_curr_setup[i] = int(input(f"Podaj ustawienie rotora chi{i+1} (liczba w zakresie 0-{rotor_chi_len[i]}): "))
    
    # ============================================================== #
    # ^ # ODKOMENTOWYWAĆ NA WŁASNĄ ODPOWIEDZIALNOŚĆ - MĘCZĄCE!!! # ^ #
    # ============================================================== #

    rotor_psi_init_setup = copy.deepcopy(rotor_psi_curr_setup)
    rotor_mu_init_setup = copy.deepcopy(rotor_mu_curr_setup)
    rotor_chi_init_setup = copy.deepcopy(rotor_chi_curr_setup)

    # pobranie od użytkownika plaintextu i przygotowanie go do pracy
    plaintext = input('Podaj plaintext (litery, bez cyfr): ')
    print(f'Plaintext: {plaintext}')
    san_pt = sanitize_text(plaintext)
    
    print(f'Przygotowany plaintext: {san_pt}')

    cyphertext = encrypt(san_pt, rotor_psi_curr_setup, rotor_mu_curr_setup, rotor_chi_curr_setup)

    print(f'Cyphertext: {cyphertext}')

    decrypted = encrypt(cyphertext, rotor_psi_init_setup, rotor_mu_init_setup, rotor_chi_init_setup)

    print(f'Decrypted: {decrypted}')

Podaj plaintext (litery, bez cyfr): TEST TEST SETST EST SETSETSETSET CXFD
Plaintext: TEST TEST SETST EST SETSETSETSET CXFD
Przygotowany plaintext: TEST TEST SETST EST SETSETSETSET CXFD
Cyphertext: V C<J%K<TXZZUKIV#GLBGJOMT@EGDIUYL<I%K
Decrypted: TEST TEST SETST EST SETSETSETSET CXFD
